In [ ]:
# Laden benötigter Bibliotheken
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy import optimize

from datetime import datetime
from requests import get

from collections import defaultdict

# Initialisierung einiger Visualisierungsparameter
plt.rcParams.update({'font.size': 14})
plt.rcParams.update({'figure.figsize': (10, 6)})
plt.rcParams.update({'lines.linewidth': 2})
plt.rcParams.update({'axes.linewidth': 2})
plt.rcParams.update({'xtick.major.width': 2})
plt.rcParams.update({'ytick.major.width': 2})

In [ ]:
# Implementierung generischer SIR, SIRD und SEIR Modelle

def sir_model(y, t, N, beta, gamma):
    S, I, R = y
    dS_dt = -beta * S * I / N
    dI_dt = beta * S * I / N - gamma * I
    dR_dt = gamma * I
    return([dS_dt, dI_dt, dR_dt])
  
def sird_model(y, t, N, beta, gamma, delta):
    S, I, R, D = y
    dS_dt = -beta * S * I / N
    dI_dt = beta * S * I / N - gamma * I - delta * I
    dR_dt = gamma * I
    dD_dt = delta * I
    return([dS_dt, dI_dt, dR_dt, dD_dt])
  
def seir_model(y, t, N, beta, gamma, alpha):
    S, E, I, R = y
    dS_dt = -beta * S * I / N
    dE_dt = beta * S * I / N - alpha * E
    dI_dt = alpha * E - gamma * I
    dR_dt = gamma * I
    return [dS_dt, dE_dt, dI_dt, dR_dt]

def calc_sir_model(days, N, I0, R0, beta, gamma):
    S0 = N - I0 - R0
    y0 = [S0, I0, R0]
    t = np.linspace(0, days, days)
    ret = odeint(sir_model, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T
    return S, I, R
  
def calc_sird_model(days, N, I0, R0, beta, gamma, delta):
    S0 = N - I0 - R0
    D0 = 0
    y0 = [S0, I0, R0, D0]
    t = np.linspace(0, days, days)
    ret = odeint(sird_model, y0, t, args=(N, beta, gamma, delta))
    S, I, R, D = ret.T
    return S, I, R, D
  
def calc_seir_model(days, N, I0, R0, beta, gamma, alpha): 
    S0 = N - I0 - R0
    E0 = 0
    y0 = [S0, E0, I0, R0]
    t = np.linspace(0, days, days)
    ret = odeint(seir_model, y0, t, args=(N, beta, gamma, alpha))
    S, E, I, R = ret.T
    return S, E, I, R

In [ ]:
# Generierung der Beispielgraphen

'''
Hier sollen nun Beispielgraphen für die drei Modelle generiert werden.
Für die bestimmung von beta und gamma werden hierfür vom RKI und dem WHO gemeldete Daten verwendet. 
Die WHO gab für eine durschnittliche Krankheitsdauer 10 Tage an, das RKI für die Basisreproduktionszahl R0 des Wildtyps 3.
Somit ergibt sich für die Parameter:

  gamma = 1/10 = 0.1
  beta = R0 * gamma = 3 * 0.1 = 0.3
'''

days = 200 # Anzahl der Tage
N = 5932654 # Einwohnerzahl von Dänemark
I0 = 1
R0 = 0
S0 = N - I0 - R0

gamma = 1/10
R = 3.3
beta = R * gamma

y0 = [S0, I0, R0]
t = np.linspace(0, days, days)

S, I, R = calc_sir_model(days, N, I0, R0, beta, gamma)

S_percent = S / N * 100
I_percent = I / N * 100
R_percent = R / N * 100


plt.plot(t, S_percent, 'b', label='Susceptible (%)')
plt.plot(t, I_percent, 'r', label='Infected (%)')
plt.plot(t, R_percent, 'g', label='Recovered (%)')
plt.xlabel('Time (days)')
plt.ylabel('Percentage of population')
plt.title('SIR Model')
plt.legend()
plt.savefig('SIR_Model.png')
plt.show()


# SIRD Model
delta = 0.018 / 10

D0 = 0
y0 = [S0, I0, R0, D0]

ret = odeint(sird_model, y0, t, args=(N, beta, gamma, delta))
S, I, R, D = ret.T

S_percent = S / N * 100
I_percent = I / N * 100
R_percent = R / N * 100
D_percent = D / N * 100

plt.plot(t, S_percent, 'b', label='Susceptible (%)')
plt.plot(t, I_percent, 'r', label='Infected (%)')
plt.plot(t, R_percent, 'g', label='Recovered (%)')
plt.plot(t, D_percent, 'k', label='Dead (%)')
plt.xlabel('Time (days)')
plt.ylabel('Percentage of population')
plt.title('SIRD Model')
plt.legend()
plt.savefig('SIRD_Model.png')
plt.show()


# SEIR Model
'''
Für das SEIR Modell werden die Parameter beta, alpha und gamma benötigt.
Für alpha wird ein Wert von 1/5,8 angenommen, da die durchschnittliche Inkubationszeit 5,8 Tage beträgt. TODO: Quelle
'''

alpha = 1/5.8
gamma = 1/10

E0 = 0
y0 = [S0, E0, I0, R0]

ret = odeint(seir_model, y0, t, args=(N, beta, gamma, alpha))
S, E, I, R = ret.T

S_percent = S / N * 100
E_percent = E / N * 100
I_percent = I / N * 100
R_percent = R / N * 100

# Plot the data
plt.plot(t, S_percent, 'b', label='Susceptible (%)')
plt.plot(t, E_percent, 'y', label='Exposed (%)')
plt.plot(t, I_percent, 'r', label='Infected (%)')
plt.plot(t, R_percent, 'g', label='Recovered (%)')
plt.xlabel('Time (days)')
plt.ylabel('Percentage of population')
plt.title('SEIR Model')
plt.legend()
plt.savefig('SEIR_Model.png')
plt.show()


## Laden der länderspezifischen Covid-19 Daten

Zum Fitten und Testen der im Laufe der Ausarbeitung implementierten Modelle möchte ich die gemeldeten COVID-19 Daten Dänemarks und des Vereinigten Königreichs nutzen. Diese werden von den Regierungen der jeweiligen Länder zur Verfügung gestellt. Um eine bessere Verfügbarkei der Daten und Leserlichkeit des Codes zu gewährleisten habe ich diese heruntergeladen, vorverarbeitet und auf einen eigenen Server hochgeladen, von welchem ich sie im folgenden ziehe.

Die Bevölkerungszahlen habe ich den jeweiligen statistischen Bundesämtern entnommen.

In [ ]:
UK_DATA_NAME = "uk_data.csv"
DENMARK_DATA_NAME = "denmark_data.csv"
HOST_NAME = "modellierung.mulorz.com"

MAX_DATE = datetime.strptime("2020-08-01", "%Y-%m-%d")

POPULATION_DENMARK = 5932654
POPULATION_UK = 67886011

def fetch_data(url):
  res = get(url)
  if res.status_code == 200:
    data_string = res.content.decode('ISO-8859-1')
    return data_string
  
def get_uk_data():
  data_url = f"http://{HOST_NAME}/{UK_DATA_NAME}"
  data = fetch_data(data_url)
  data = preprocess_uk_data(data)
  return data

def preprocess_uk_data(data):
  data = data.split("\n")
  data = [line.split(";") for line in data][1:]  
  max_date_index = next(
    (i for i, line in enumerate(data) if datetime.strptime(line[0], "%Y-%m-%d") >= MAX_DATE),
    None
  )
  data = data[:max_date_index if max_date_index else len(data)]
  merged_list = [{'date': datum[0], 'newCases': int(datum[1])} for datum in data]
  
  return merged_list

def get_denmark_data():
  data_url = f"http://{HOST_NAME}/{DENMARK_DATA_NAME}"
  data = fetch_data(data_url)
  data = preprocess_denmark_data(data)
  return data

def preprocess_denmark_data(data):
  data = data.split("\n")
  data = data[1:-1]
  dates = [line.split(";")[2] for line in data]
  cases = [line.split(";")[4] for line in data]
  
  max_date_index = next(
    (i for i, date in enumerate(dates) if datetime.strptime(date, "%Y-%m-%d") >= MAX_DATE),
    None
  )
  
  dates = dates[:max_date_index if max_date_index else len(dates)]
  cases = cases[:max_date_index if max_date_index else len(cases)]
  aggregated_cases = defaultdict(int)
  
  for date, case in zip(dates, cases):
    aggregated_cases[date] += int(case)
  merged_list = [{'date': date, 'newCases': new_cases} for date, new_cases in aggregated_cases.items()]
  
  return merged_list

def add_up_cases(cases, days=10):
  cum_cases = []
  for i in range(len(cases)):
    entry_data = cases[i]['date']
    entry_cases = 0
    for j in range(1, days+1):
      if i-j >= 0:
        entry_cases += cases[i-j]['newCases']
    cum_cases.append({'date': entry_data, 'totalCases': entry_cases})
  return cum_cases

def get_index_of_first_case(data):
  for i in range(len(data)):
    if data[i]['totalCases'] > 0:
      return i

uk_data = get_uk_data() # Dict mit Datum und Anzahl neuer Fälle. Keys: "date", "newCases"
denmark_data = get_denmark_data()

cum_uk_data = add_up_cases(uk_data)
cum_denmark_data = add_up_cases(denmark_data)

In [ ]:
# Visualisieren der Daten

def visualize_data(dates, denmark_cases, uk_cases):
  plt.figure(figsize=(20, 12))
  plt.plot(dates, denmark_cases, label='Denmark', marker='o')
  plt.plot(dates, uk_cases, label='UK', marker='o')
  plt.xlabel('Date')
  plt.ylabel('Number of Cases per 1000')
  plt.title('COVID-19 Cases per 1000 in Denmark and UK')
  ax = plt.gca()
  ax.set_xticks(ax.get_xticks()[::7])
  plt.xticks(rotation=45)
  plt.legend()
  plt.tight_layout()
  plt.show()
  
def show_data(uk_data, denmark_data, case_type='newCases'):
  combined_data = defaultdict(dict)

  for entry in denmark_data:
    date = datetime.strptime(entry['date'], "%Y-%m-%d")
    if date < MAX_DATE:
      combined_data[entry['date']]['date'] = entry['date']
      combined_data[entry['date']]['denmark'] = entry[case_type] / POPULATION_DENMARK * 1000
      
  for entry in uk_data:
    date = datetime.strptime(entry['date'], "%Y-%m-%d")
    if date < MAX_DATE:
      combined_data[entry['date']]['date'] = entry['date']
      combined_data[entry['date']]['uk'] = entry[case_type] / POPULATION_UK * 1000

  merged_list = list(combined_data.values())
  merged_list.sort(key=lambda x: x['date'])

  dates = [entry['date'] for entry in merged_list]
  denmark_cases = [entry.get('denmark', 0) for entry in merged_list] 
  uk_cases = [entry.get('uk', 0) for entry in merged_list] 
  visualize_data(dates, denmark_cases, uk_cases)
  
show_data(uk_data, denmark_data)


# Add up cases for the last 10 days
def add_up_cases(cases, days=10):
  cum_cases = []
  for i in range(len(cases)):
    entry_data = cases[i]['date']
    entry_cases = 0
    for j in range(1, days+1):
      if i-j >= 0:
        entry_cases += cases[i-j]['newCases']
    cum_cases.append({'date': entry_data, 'totalCases': entry_cases})
  return cum_cases


def get_index_of_first_case(data):
  for i in range(len(data)):
    if data[i]['totalCases'] > 0:
      return i



In [ ]:
## Vergleich SIR, SIRD und SEIR Modelle mit den Daten von Dänemark

gamma = 1/10
R = 3.3
beta = R * gamma
delta = 0.018 / 10
alpha = 1/5.8

cum_denmark_cases = add_up_cases(denmark_data)
t = np.linspace(0, len(cum_denmark_cases) - 1, len(cum_denmark_cases) - 1)
I0 = 1
R0 = 0
index_of_first_case = get_index_of_first_case(cum_denmark_cases)
spliced_cases = cum_denmark_cases[index_of_first_case:]
total_cases = [datum['totalCases'] for datum in spliced_cases]
S,I,R = calc_sir_model(len(total_cases), POPULATION_DENMARK, I0, R0, 0.3, gamma)
S2, I2, R2, D2 = calc_sird_model(len(total_cases), POPULATION_DENMARK, I0, R0, 0.3, gamma, delta)
S3, E3, I3, R3 = calc_seir_model(len(total_cases), POPULATION_DENMARK, I0, R0, 0.3, gamma, alpha)

plt.plot(I, label='SIR Infected')
plt.plot(I2, label='SIRD Infected')
plt.plot(I3, label='SEIR Infected')
plt.plot(t, total_cases, label='Reported Cases in Denmark')
plt.xlabel('Days')
plt.ylabel('Number of Cases')
plt.title('COVID-19 Cases in Denmark compared to SIR, SIRD and SEIR Models')
ax = plt.gca()
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig('COVID-19_Cases_Denmark_comparison.png')
plt.show()


def calculate_mse(data, I):
  total_cases = [datum['totalCases'] for datum in data]
  return np.mean((np.array(total_cases) - np.array(I)) ** 2)



mse_sir = calculate_mse(spliced_cases, I)
mse_sird = calculate_mse(spliced_cases, I2)
mse_seir = calculate_mse(spliced_cases, I3)

print(f'MSE SIR: {mse_sir}')
print(f'MSE SIRD: {mse_sird}')
print(f'MSE SEIR: {mse_seir}')

In [ ]:
# Fitten des SIR-Modells an gemeldete COVID-19 Fälle

POPULATION = 0

# Add up cases for the last 10 days
def add_up_cases(cases, days=10):
  cum_cases = []
  for i in range(len(cases)):
    entry_data = cases[i]['date']
    entry_cases = 0
    for j in range(1, days+1):
      if i-j >= 0:
        entry_cases += cases[i-j]['newCases']
    cum_cases.append({'date': entry_data, 'totalCases': entry_cases})
  return cum_cases

def sir_model_fit(t, beta, gamma):
    I0 = 1
    R0 = 0
    S0 = POPULATION - I0 - R0
    y0 = S0, I0, R0
    ret = odeint(sir_model, y0, t, args=(POPULATION, beta, gamma))
    return ret[:, 1]

def get_index_of_first_case(data):
  for i in range(len(data)):
    if data[i]['totalCases'] > 0:
      return i
    
# Visualisierung der Fitting-Ergebnisse
def visualize_fitting(t, total_cases_data, beta, gamma, I0, S0, R0, population, country):
  y0 = S0, I0, R0
  ret = odeint(sir_model, y0, t, args=(population, beta, gamma))
  S, I, R = ret.T

  plt.plot(t, total_cases_data, label='Real Data')
  plt.plot(t, I, label='Fitted SIR Model')
  plt.xlabel('Time (days)')
  plt.ylabel('Number of Cases')
  plt.title(f"Fitting SIR Model to {country} Data")
  plt.legend()
  plt.savefig(f'Fitting_{country}_SIR_Model.png')
  #plt.show()
  
  return ret
  
def fit_data(cum_cases_data, population, country):
  global POPULATION
  
  index_of_first_case = get_index_of_first_case(cum_cases_data)
  spliced_cases = cum_cases_data[index_of_first_case:]
  total_cases = [datum['totalCases'] for datum in spliced_cases]
  POPULATION = population
  I0 = total_cases[0]
  R0 = 0
  S0 = population - I0 - R0
  
  t = np.linspace(0, len(total_cases), len(total_cases))

  popt, _pcov = optimize.curve_fit(sir_model_fit, t, total_cases)

  beta_opt, gamma_opt = popt
  
  print(f"Optimal Beta: {beta_opt}, Optimal Gamma: {gamma_opt}")
  return visualize_fitting(t, total_cases, beta_opt, gamma_opt, I0, S0, R0, population, country)
  
cum_denmark_cases = add_up_cases(denmark_data)
cum_uk_cases = add_up_cases(uk_data)

ret_denmark = fit_data(cum_denmark_cases, POPULATION_DENMARK, "Denmark")
ret_uk = fit_data(cum_uk_cases, POPULATION_UK, "UK")

# Berechnen der Fehler
_, i_denmark, _ = ret_denmark.T
_, i_uk, _ = ret_uk.T
print(f"len i_denmark: {len(i_denmark)}, cum_denmark_cases: {len(cum_denmark_cases)}")
print(f"len i_uk: {len(i_uk)}, cum_uk_cases: {len(cum_uk_cases)}")
mse_denmark = calculate_mse(cum_denmark_cases[:len(i_denmark)], i_denmark)
mse_uk = calculate_mse(cum_uk_cases[:len(i_uk)], i_uk)

print(f'MSE Denmark: {mse_denmark}')
print(f'MSE UK: {mse_uk}')



In [ ]:
# Fitten des SIRD-Modells mit nicht-linearem L an die Daten von Dänemark

def calcL(t, tc, r, lockdown_rampup_duration, p):
  if t < tc:
    return r
  elif t >= tc and t < tc + lockdown_rampup_duration:
    power = (p - 1) / lockdown_rampup_duration * (t - tc) + 1
    return r ** power
  else:
    return r ** p


def sird_model_l(y, t, beta, gamma, delta, p):
  S = y[0]
  I = y[1]
  R = y[2]
  D = y[3]
  N = S + I + R + D
  
  lockdown_rampup_duration = 10
  tc = 19
  
  dS_dt = -beta * calcL(t, tc, S/N, lockdown_rampup_duration, p) * I
  dI_dt = beta * calcL(t, tc, S/N, lockdown_rampup_duration, p) * I - (gamma + delta) * I
  dR_dt = gamma * I
  dD_dt = delta * I
  return([dS_dt, dI_dt, dR_dt, dD_dt])

def sird_model_l_wrapper(t, beta, gamma, delta, p):
  N = POPULATION_DENMARK
  S = N - 1
  I = 1
  R = 0
  D = 0
  return odeint(sird_model_l, [S, I, R, D], t, args=(beta, gamma, delta, p))[:,1]


def sird_model_l_fit():
  t = np.linspace(0, 94, 94)
  data = [datum['totalCases'] for datum in cum_denmark_cases][:94]
  popt, pcov = optimize.curve_fit(sird_model_l_wrapper, t, data, p0=[0.23, 0.05, 0.01, 1.2], bounds=([0, 0, 0, 1.2], [10, 1, 1, np.iinfo(np.int32).max]), maxfev=10000)
  return popt, pcov


ret, pcov = sird_model_l_fit()
print(f"Optimale Parameter: {ret}")
print(f"Kovarianzmatrix: {pcov}")

ret = odeint(sird_model_l, [POPULATION_DENMARK, 1, 0, 0], np.linspace(0, 94, 94), args=(ret.T[0], ret.T[1], ret.T[2], ret.T[3]))
S, I, R, D = ret.T
plt.plot(I, label="Infected")
plt.plot([datum['totalCases'] for datum in cum_denmark_cases][:94], label="Reported Cases")
plt.title("Fitting SIRD Model with non-linear L to Denmark Data")
plt.legend()
plt.savefig("Fitting_SIRDL_Model_Denmark.png")
plt.show()


error = np.mean((np.array([datum['totalCases'] for datum in cum_denmark_cases][:94]) - np.array(I)) ** 2)
print(f"MSE: {error}")

In [ ]:
# Fitten des SIRD-Modells mit nicht-linearem L an die Daten von UK

def calcL(t, tc, r, lockdown_rampup_duration, p):
  if t < tc:
    return r
  elif t >= tc and t < tc + lockdown_rampup_duration:
    power = (p - 1) / lockdown_rampup_duration * (t - tc) + 1
    return r ** power
  else:
    return r ** p


def sird_model_l(y, t, beta, gamma, delta, p):
  S = y[0]
  I = y[1]
  R = y[2]
  D = y[3]
  N = S + I + R + D
  
  lockdown_rampup_duration = 10
  tc = 52
  
  dS_dt = -beta * calcL(t, tc, S/N, lockdown_rampup_duration, p) * I
  dI_dt = beta * calcL(t, tc, S/N, lockdown_rampup_duration, p) * I - (gamma + delta) * I
  dR_dt = gamma * I
  dD_dt = delta * I
  return([dS_dt, dI_dt, dR_dt, dD_dt])

def sird_model_l_wrapper(t, beta, gamma, delta, p):
  N = POPULATION_UK
  S = N - 1
  I = 1
  R = 0
  D = 0
  return odeint(sird_model_l, [S, I, R, D], t, args=(beta, gamma, delta, p))[:,1]


def sird_model_l_fit():
  t = np.linspace(0, 121, 121)
  data = [datum['totalCases'] for datum in cum_uk_cases][:121]
  popt, pcov = optimize.curve_fit(sird_model_l_wrapper, t, data, p0=[0.23, 0.05, 0.01, 1.2], bounds=([0, 0, 0, 1.2], [10, 1, 1, np.iinfo(np.int32).max]), maxfev=10000)
  return popt, pcov

  

ret, pcov = sird_model_l_fit()
print(f"Optimale Parameter: {ret}")
print(f"Kovarianzmatrix: {pcov}")

ret = odeint(sird_model_l, [POPULATION_UK, 1, 0, 0], np.linspace(0, 121, 121), args=(ret.T[0], ret.T[1], ret.T[2], ret.T[3]))
S, I, R, D = ret.T
plt.plot(I, label="Infected")
plt.plot([datum['totalCases'] for datum in cum_uk_cases][:121], label="Reported Cases")
plt.title("Fitting SIRD Model with non-linear L to UK Data")
plt.legend()
plt.savefig("Fitting_SIRDL_Model_UK.png")
plt.show()

error = np.mean((np.array([datum['totalCases'] for datum in cum_uk_cases][:121]) - np.array(I)) ** 2)
print(f"MSE: {error}")

In [ ]:
# SEAQR für UK

def seaqr_model(y, t, beta, gamma, sigma, alpha, kappa):
  S, E, A, Q, R = y
  N = S + E + A + Q + R
  
  dS_dt = -beta * S * A / N
  dE_dt = beta * A * S / N  - gamma * E - sigma * E
  dA_dt = gamma * E - alpha * A
  dQ_dt = sigma * E - kappa * Q
  dR_dt = kappa * Q + alpha * A
  return([dS_dt, dE_dt, dA_dt, dQ_dt, dR_dt])

def seaqr_model_wrapper(t, beta, gamma, sigma, alpha, kappa):
  N = POPULATION_UK
  S0 = N - 1
  E0 = 0
  A0 = 1
  Q0 = 0
  R0 = 0
  y0 = [S0, E0, A0, Q0, R0]
  t = np.linspace(0, 100, 100)
  result = odeint(seaqr_model, y0, t, args=(beta, gamma, sigma, alpha, kappa))
  return result[:, 2] + result[:, 3]

def seaqr_model_fit():
  t = np.linspace(0, 100, 100)
  data = [datum['totalCases'] for datum in cum_uk_cases][:100]
  popt, pcov = optimize.curve_fit(seaqr_model_wrapper, t, data, p0=[0.23, 1/5.8, 1/5.8, 0.1, 0.1], bounds=([0, 0, 0, 0, 0], [10, 1, 1, 1, 1]), maxfev=100000)
  return popt, pcov

beta = 0.23
gamma = 1/5.8
sigma = 1/5.8
alpha = 0.1
kappa = 0.1

def calc_seaqr_model(days, beta, gamma, sigma, alpha, kappa):
    S0 = 100
    E0 = 0
    A0 = 10
    Q0 = 0
    R0 = 0
    y0 = [S0, E0, A0, Q0, R0]
    t = np.linspace(0, 100, 100)
    ret = odeint(seaqr_model, y0, t, args=(beta, gamma, sigma, alpha, kappa))
    S, E, A, Q, R = ret.T
    return S, E, A, Q, R
  
ret, pcov = seaqr_model_fit()
print(f"Optimale Parameter: {ret}")
print(f"Kovarianzmatrix: {pcov}")

ret = odeint(seaqr_model, [POPULATION_UK, 0, 2, 0, 0], np.linspace(0, 121, 121), args=(ret.T[0], ret.T[1], ret.T[2], ret.T[3], ret.T[4]))
S, E, A, Q, R = ret.T
I = A + Q
plt.plot(I, label="Infected")
plt.plot([datum['totalCases'] for datum in cum_uk_cases][:121], label="Reported Cases")
plt.title("Fitting SEAQR Model to UK Data")
plt.legend()
plt.savefig("Fitting_SEAQR_Model_UK.png")
plt.show()

# Berechnen des mittleren quadratischen Fehlers
error = np.mean((np.array([datum['totalCases'] for datum in cum_uk_cases][:121]) - np.array(I)) ** 2)
print(f"MSE: {error}")

In [ ]:
# SEAQR Modell gefittet auf Daten Dänemarks

def seaqr_model(y, t, beta, gamma, sigma, alpha, kappa):
  S, E, A, Q, R = y
  N = S + E + A + Q + R
  
  dS_dt = -beta * S * A / N
  dE_dt = beta * A * S / N  - gamma * E - sigma * E
  dA_dt = gamma * E - alpha * A
  dQ_dt = sigma * E - kappa * Q
  dR_dt = kappa * Q + alpha * A
  return([dS_dt, dE_dt, dA_dt, dQ_dt, dR_dt])

def seaqr_model_wrapper(t, beta, gamma, sigma, alpha, kappa):
  N = POPULATION_DENMARK
  S0 = N - 1
  E0 = 0
  A0 = 1
  Q0 = 0
  R0 = 0
  y0 = [S0, E0, A0, Q0, R0]
  t = np.linspace(0, 45, 45)
  result = odeint(seaqr_model, y0, t, args=(beta, gamma, sigma, alpha, kappa))
  return result[:, 2] + result[:, 3]

def seaqr_model_fit():
  t = np.linspace(0, 45, 45)
  data = [datum['totalCases'] for datum in cum_denmark_cases][:45]
  popt, _pcov = optimize.curve_fit(seaqr_model_wrapper, t, data, bounds=([0, 0, 0, 0, 0], [10, 1, 1, 1, 1]), maxfev=10000)
  return popt

beta = 0.23
gamma = 1/5.8
sigma = 1/5.8
alpha = 0.1
kappa = 0.1

def calc_seaqr_model(days, beta, gamma, sigma, alpha, kappa):
    S0 = 100
    E0 = 0
    A0 = 10
    Q0 = 0
    R0 = 0
    y0 = [S0, E0, A0, Q0, R0]
    t = np.linspace(0, 94, 94)
    ret = odeint(seaqr_model, y0, t, args=(beta, gamma, sigma, alpha, kappa))
    S, E, A, Q, R = ret.T
    return S, E, A, Q, R
  
ret = seaqr_model_fit()
print(f"Optimale Parameter: {ret}")

ret = odeint(seaqr_model, [POPULATION_DENMARK, 0, 2, 0, 0], np.linspace(0, 94, 94), args=(ret.T[0], ret.T[1], ret.T[2], ret.T[3], ret.T[4]))
S, E, A, Q, R = ret.T
I = A + Q
plt.plot(I, label="Infected (A + Q)")
plt.plot([datum['totalCases'] for datum in cum_denmark_cases][:94], label="Reported Cases")
plt.title("Fitting SEAQR Model to Denmark Data")
plt.legend()
plt.savefig("Fitting_SEAQR_Model_DENMARK.png")
plt.show()

# Berechnen des mittleren quadratischen Fehlers
error = np.mean((np.array([datum['totalCases'] for datum in cum_denmark_cases][:94]) - np.array(I)) ** 2)
print(f"MSE: {error}")

In [ ]:
# SEAQR mit nicht-linearem L für UK

def calcL(t, tc, r, lockdown_rampup_duration, p):
  if t < tc:
    return r
  elif t >= tc and t < tc + lockdown_rampup_duration:
    power = (p - 1) / lockdown_rampup_duration * (t - tc) + 1
    return r ** power
  else:
    return r ** p

def seaqr_model(y, t, beta, gamma, sigma, alpha, kappa, p):
  S, E, A, Q, R = y
  N = S + E + A + Q + R
  
  lockdown_rampup_duration = 10
  tc = 19 # Lockdown start (https://www.regeringen.dk/nyheder/2020/pressemoede-11-marts-i-spejlsalen/)
  
  dS_dt = -beta * calcL(t, tc, S/N, lockdown_rampup_duration, p) * A
  dE_dt = beta * A * calcL(t, tc, S/N, lockdown_rampup_duration, p)  - gamma * E - sigma * E
  dA_dt = gamma * E - alpha * A
  dQ_dt = sigma * E - kappa * Q
  dR_dt = kappa * Q + alpha * A
  return([dS_dt, dE_dt, dA_dt, dQ_dt, dR_dt])

def seaqr_model_wrapper(t, beta, gamma, sigma, alpha, kappa, p):
  N = POPULATION_DENMARK
  S0 = N - 1
  E0 = 0
  A0 = 1
  Q0 = 0
  R0 = 0
  y0 = [S0, E0, A0, Q0, R0]
  t = np.linspace(0, 94, 94)
  result = odeint(seaqr_model, y0, t, args=(beta, gamma, sigma, alpha, kappa, p))
  return result[:, 2] + result[:, 3]

def seaqr_model_fit():
  t = np.linspace(0, 94, 94)
  data = [datum['totalCases'] for datum in cum_denmark_cases][:94]
  popt, _pcov = optimize.curve_fit(seaqr_model_wrapper, t, data, bounds=([0, 0, 0, 0, 0, 1], [10, 1, 1, 1, 1, 10000]), maxfev=10000)
  return popt, _pcov

beta = 0.23
gamma = 1/5.8
sigma = 1/5.8
alpha = 0.1
kappa = 0.1

def calc_seaqr_model(days, beta, gamma, sigma, alpha, kappa, p):
    S0 = 100
    E0 = 0
    A0 = 10
    Q0 = 0
    R0 = 0
    y0 = [S0, E0, A0, Q0, R0]
    t = np.linspace(0, 94, 94)
    ret = odeint(seaqr_model, y0, t, args=(beta, gamma, sigma, alpha, kappa, p))
    S, E, A, Q, R = ret.T
    return S, E, A, Q, R
  
ret, pcov = seaqr_model_fit()
print(f"Optimale Parameter: {ret}")
print(f"pcov: {pcov}")

ret = odeint(seaqr_model, [POPULATION_DENMARK, 0, 2, 0, 0], np.linspace(0, 94, 94), args=(ret.T[0], ret.T[1], ret.T[2], ret.T[3], ret.T[4], ret.T[5]))
S, E, A, Q, R = ret.T
I = A + Q
plt.plot(I, label="Infected (A + Q)")
plt.plot([datum['totalCases'] for datum in cum_denmark_cases][:94], label="Reported Cases")
plt.title("Fitted SEAQR+L Model to Denmark Data")
plt.legend()
plt.savefig("Fitted_SEAQRL_Model_DENMARK.png")
plt.show()

# Berechnen des mittleren quadratischen Fehlers
error = np.mean((np.array([datum['totalCases'] for datum in cum_denmark_cases][:94]) - np.array(I)) ** 2)
print(f"MSE: {error}")

In [ ]:
# SEAQR mit nicht-linearem L für UK

def calcL(t, tc, r, lockdown_rampup_duration, p):
  if t < tc:
    return r
  elif t >= tc and t < tc + lockdown_rampup_duration:
    power = (p - 1) / lockdown_rampup_duration * (t - tc) + 1
    return r ** power
  else:
    return r ** p

def seaqr_model(y, t, beta, gamma, sigma, alpha, kappa, p):
  S, E, A, Q, R = y
  N = S + E + A + Q + R
  
  lockdown_rampup_duration = 10
  tc = 52 # Lockdown Start (https://www.legislation.gov.uk/uksi/2020/350/made)
  
  dS_dt = -beta * calcL(t, tc, S/N, lockdown_rampup_duration, p) * A
  dE_dt = beta * A * calcL(t, tc, S/N, lockdown_rampup_duration, p)  - gamma * E - sigma * E
  dA_dt = gamma * E - alpha * A
  dQ_dt = sigma * E - kappa * Q
  dR_dt = kappa * Q + alpha * A
  return([dS_dt, dE_dt, dA_dt, dQ_dt, dR_dt])

def seaqr_model_wrapper(t, beta, gamma, sigma, alpha, kappa, p):
  N = POPULATION_UK
  S0 = N - 1
  E0 = 0
  A0 = 1
  Q0 = 0
  R0 = 0
  y0 = [S0, E0, A0, Q0, R0]
  t = np.linspace(0, 121, 121)
  result = odeint(seaqr_model, y0, t, args=(beta, gamma, sigma, alpha, kappa, p))
  return result[:, 2] + result[:, 3]

def seaqr_model_fit():
  t = np.linspace(0, 121, 121)
  data = [datum['totalCases'] for datum in cum_uk_cases][:121]
  popt, _pcov = optimize.curve_fit(seaqr_model_wrapper, t, data, p0=[0.23, 1/5.8, 1/5.8, 0.1, 0.1, 1], bounds=([0, 0, 0, 0, 0, 1], [10, 1, 1, 1, 1, 10000]), maxfev=10000)
  return popt, _pcov

beta = 0.23
gamma = 1/5.8
sigma = 1/5.8
alpha = 0.1
kappa = 0.1

def calc_seaqr_model(days, beta, gamma, sigma, alpha, kappa, p):
    S0 = 100
    E0 = 0
    A0 = 10
    Q0 = 0
    R0 = 0
    y0 = [S0, E0, A0, Q0, R0]
    t = np.linspace(0, 121, 121)
    ret = odeint(seaqr_model, y0, t, args=(beta, gamma, sigma, alpha, kappa, p))
    S, E, A, Q, R = ret.T
    return S, E, A, Q, R
  
ret, pcov = seaqr_model_fit()
print(f"Optimale Parameter: {ret}")
print(f"Covariance Matrix: {pcov}")

ret = odeint(seaqr_model, [POPULATION_DENMARK, 0, 2, 0, 0], np.linspace(0, 121, 121), args=(ret.T[0], ret.T[1], ret.T[2], ret.T[3], ret.T[4], ret.T[5]))
S, E, A, Q, R = ret.T
I = A + Q
plt.plot(I, label="Infected")
plt.plot([datum['totalCases'] for datum in cum_uk_cases][:121], label="Reported Cases")
plt.title("Fitted SEAQR+L Model to UK Data")
plt.legend()
plt.savefig("Fitting_SEAQRL_Model_UK.png")
plt.show()

# Berechnen des mittleren quadratischen Fehlers
error = np.mean((np.array([datum['totalCases'] for datum in cum_uk_cases][:121]) - np.array(I)) ** 2)
print(f"MSE: {error}")